In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
crimes = pd.read_csv('/Users/alvin/Downloads/CrimesSmall.csv')

In [3]:
# get list of unique districts 1-25
districts = crimes['District'].unique().tolist()
districts

[10, 9, 18, 5, 16, 19, 2, 17, 1, 15, 8, 22, 6, 14, 12, 7, 3, 11, 25, 4, 20, 24]

In [4]:
crime_types = crimes['Primary Type'].unique().tolist()
crime_types

['ASSAULT',
 'DECEPTIVE PRACTICE',
 'CRIMINAL DAMAGE',
 'THEFT',
 'BATTERY',
 'NARCOTICS',
 'MOTOR VEHICLE THEFT',
 'PUBLIC PEACE VIOLATION',
 'ROBBERY',
 'OTHER OFFENSE',
 'OFFENSE INVOLVING CHILDREN',
 'CRIMINAL TRESPASS',
 'PROSTITUTION',
 'BURGLARY',
 'WEAPONS VIOLATION',
 'ARSON',
 'LIQUOR LAW VIOLATION',
 'SEX OFFENSE',
 'KIDNAPPING',
 'CRIM SEXUAL ASSAULT',
 'INTERFERENCE WITH PUBLIC OFFICER',
 'GAMBLING',
 'INTIMIDATION']

In [5]:
violent_crimes = ['BATTERY','ASSAULT','CRIM SEXUAL ASSAULT', 'ARSON','HOMICIDE']
felonies = ['HUMAN TRAFFICKING', 'HOMICIDE','ARSON','KIDNAPPING','BURGLARY','ROBBERY','BATTERY','STALKING','SEX OFFENSE']

In [6]:
violents = []
for crime in violent_crimes:
    violents.append( crimes.loc[ crimes['Primary Type'] == crime] )

In [7]:
df_violent = violents[0]
for i in range(1,len(violents)):
    df_violent.append(violents[i])

In [8]:
df_violent;

In [9]:
import pandas as pd

In [10]:
#crimes = pd.read_csv('~/Downloads/cleaned_chicago_crime_data.csv')

In [11]:
crimes.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,7956475,HT188097,3/4/2011 10:52,025XX S KILDARE AVE,554,ASSAULT,AGG PO HANDS NO/MIN INJURY,"SCHOOL, PUBLIC, BUILDING",True,False,...,22.0,30.0,08A,1148345.0,1886897.0,2011,2/10/2018 15:50,41.845586,-87.731082,"(41.84558589, -87.731082337)"
1,7956476,HT185132,3/4/2011 12:00,024XX W 45TH PL,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,RESIDENCE,False,False,...,12.0,58.0,11,1160803.0,1874437.0,2011,2/10/2018 15:50,41.811145,-87.685707,"(41.811145384, -87.685707211)"
2,7956477,HT187881,3/4/2011 9:00,023XX N CLYBOURN AVE,1310,CRIMINAL DAMAGE,TO PROPERTY,RESIDENCE,False,False,...,32.0,7.0,14,1165097.0,1915872.0,2011,2/10/2018 15:50,41.924757,-87.668781,"(41.924756753, -87.668780552)"
3,7956479,HT187939,3/3/2011 8:00,003XX W 100TH ST,890,THEFT,FROM BUILDING,RESIDENTIAL YARD (FRONT/BACK),False,False,...,9.0,49.0,6,1175827.0,1838618.0,2011,2/10/2018 15:50,41.712530,-87.631672,"(41.712529985, -87.631672138)"
4,7956480,HT187986,3/4/2011 10:35,065XX W BRYN MAWR AVE,460,BATTERY,SIMPLE,"SCHOOL, PUBLIC, BUILDING",True,False,...,41.0,10.0,08B,1131583.0,1936817.0,2011,2/10/2018 15:50,41.982879,-87.791441,"(41.98287878, -87.791440835)"


In [12]:
crimes.index = crimes['ID'] # make the index the actual case number
# remove columns that are unnecessary for the analysis and would probably just slow our code down
crimes = crimes.drop(['Beat', 'FBI Code','Updated On', 'Case Number', 'Block', 'IUCR', 'ID', 'Unnamed: 0','Description', 'Ward', 'X Coordinate', 'Y Coordinate', 'Latitude', 'Longitude'], axis=1)

KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
crime_types = crimes['Primary Type'].unique().tolist()

### Begin cleaning data

1. Remove NaN values
2. standardize the crime type to get rid of the uppercase, standardize the non-criminal type
3. standardize the location description

There are actually no Nan values to remove! Government data is reliable I guess. The only NaN values are location, but we still know the district, so as long as we don't end up needing location later on, this is fine

In [ ]:
to_drop = crimes[crimes.isnull().any(axis=1)]
to_drop;

In [ ]:
def standardize_primary_type(string):
    # compile all non-criminal offenses into on label
    if string == 'NON-CRIMINAL (SUBJECT SPECIFIED)' or string == 'NON - CRIMINAL' or string == 'NON-CRIMINAL':
        return 'non-criminal'
    if string == 'OTHER OFFENSE':
        return 'other'
    # rename crim sexual assault to just sexual assault to make it easier to read
    if string == 'CRIM SEXUAL ASSAULT':
        return 'sexual assault'
    else:
        # everything else, make sure to lowercase it so we don't have to use caps lock lol
        return string.lower()
        

In [ ]:
def standardize_location(string):
    if type(string) == str:
        string = string.lower()
        if 'airport' in string:
            return 'airport'
    return string

In [ ]:
crimes['Primary Type'] = crimes['Primary Type'].apply(standardize_primary_type) # clean data 
crimes['location'] = crimes['Location Description'].apply(standardize_location)

In [ ]:
# be careful if you run this one, only run it once it won't work the second time 
crimes = crimes.drop(['Location Description','Community Area'],axis=1)

In [ ]:
crimes = crimes.rename(columns = {'Location': 'Coordinates'})

In [ ]:
violents = ['BATTERY','ASSAULT','CRIM SEXUAL ASSAULT', 'ARSON','HOMICIDE']

In [ ]:
violent_crimes = crimes[ (crimes['Primary Type'] == 'battery') | (crimes['Primary Type'] == 'assault') | (crimes['Primary Type'] == 'sexual assault') | (crimes['Primary Type'] == 'arson') | (crimes['Primary Type'] == 'homicide')]

In [13]:
#binomial_model = smf.glm(formula='ArrestInts ~ Type + District', data=crimes.loc[ind,:] , family=sm.families.Binomial())
binomial_model = smf.glm(formula='ArrestInts ~ Type + District', data=crimes , family=sm.families.Binomial())
binomial_results = binomial_model.fit()
binomial_results.summary()

NameError: name 'smf' is not defined

In [ ]:
sns.regplot(x = )